In [1]:
import pandas as pd

games_df = pd.read_csv('./Data/imdb-videogames.csv')
games_df = games_df.drop(games_df.columns[0], axis=1)
print(games_df.head())


                         name  \
0                  Spider-Man   
1      Red Dead Redemption II   
2          Grand Theft Auto V   
3                  God of War   
4  Uncharted 4: A Thief's End   

                                                 url    year certificate  \
0  https://www.imdb.com/title/tt5807780/?ref_=adv...  2018.0           T   
1  https://www.imdb.com/title/tt6161168/?ref_=adv...  2018.0           M   
2  https://www.imdb.com/title/tt2103188/?ref_=adv...  2013.0           M   
3  https://www.imdb.com/title/tt5838588/?ref_=adv...  2018.0           M   
4  https://www.imdb.com/title/tt3334704/?ref_=adv...  2016.0           T   

   rating   votes                                               plot  Action  \
0     9.2  20,759  When a new villain threatens New York City, Pe...    True   
1     9.7  35,703  Amidst the decline of the Wild West at the tur...    True   
2     9.5  59,986  Three very different criminals team up for a s...    True   
3     9.6  26,118  After

In [2]:
# Check what the objects are and where the null values are at
print(games_df.dtypes)
print()
print()
print(games_df.isnull().sum())


name            object
url             object
year           float64
certificate     object
rating         float64
votes           object
plot            object
Action            bool
Adventure         bool
Comedy            bool
Crime             bool
Family            bool
Fantasy           bool
Mystery           bool
Sci-Fi            bool
Thriller          bool
dtype: object


name               0
url                0
year             267
certificate    12900
rating          9203
votes           9203
plot               0
Action             0
Adventure          0
Comedy             0
Crime              0
Family             0
Fantasy            0
Mystery            0
Sci-Fi             0
Thriller           0
dtype: int64


In [3]:
# Convert 'votes' to numeric, after removing any commas
games_df['votes'] = games_df['votes'].str.replace(',', '')
games_df['votes'] = pd.to_numeric(games_df['votes'])
print(games_df.dtypes)

name            object
url             object
year           float64
certificate     object
rating         float64
votes          float64
plot            object
Action            bool
Adventure         bool
Comedy            bool
Crime             bool
Family            bool
Fantasy           bool
Mystery           bool
Sci-Fi            bool
Thriller          bool
dtype: object


Based off of whats missing we are going to create three new datasets for clustering and at the same time we are removing url and plot from all of them because they are objects/strings and wont do anything for clustering.  We will keep name in all so we can see what games are in the different clusters.

In [4]:
# Create genreonly_df by selecting 'name' and the genre columns
genreonly_df = games_df[['name', 'Action', 'Adventure', 'Comedy', 'Crime', 
'Family', 'Fantasy', 'Mystery', 'Sci-Fi', 'Thriller']].copy()

print(genreonly_df.head())
print(genreonly_df.isnull().sum())

# Get the number of rows in with_certificate_df
num_rows = genreonly_df.shape[0]

print(f"Number of rows in with_certificate_df: {num_rows}")

                         name  Action  Adventure  Comedy  Crime  Family  \
0                  Spider-Man    True       True   False  False   False   
1      Red Dead Redemption II    True       True   False   True   False   
2          Grand Theft Auto V    True      False   False   True   False   
3                  God of War    True       True   False  False   False   
4  Uncharted 4: A Thief's End    True       True   False  False   False   

   Fantasy  Mystery  Sci-Fi  Thriller  
0     True    False   False     False  
1    False    False   False     False  
2    False    False   False     False  
3    False    False   False     False  
4    False    False   False     False  
name         0
Action       0
Adventure    0
Comedy       0
Crime        0
Family       0
Fantasy      0
Mystery      0
Sci-Fi       0
Thriller     0
dtype: int64
Number of rows in with_certificate_df: 20803


Here are some of the preliminary things we hope to discover with the unsupervised clustering. <br/>
<br/>
Clustering only on the genre could be useful for understanding how games group together purely based on genre. You might discover, for example, that certain combinations of genres are particularly common. This can help you identify common trends in video game genres and can provide insights on what combination of genres are popular or less explored in the market.

In [5]:
# Check if all nulls in 'year' are in rows where 'certificate' is also null
null_year_certificate = games_df[games_df['year'].isnull() & games_df['certificate'].isnull()]

print(f"Number of rows where both 'year' and 'certificate' are null: {len(null_year_certificate)}")


Number of rows where both 'year' and 'certificate' are null: 247


In [6]:
# next dataframe will be with as many certificates as possible without nulls
# Create a new DataFrame with selected columns
with_certificate_df = games_df[['name', 'year', 'certificate', 'rating' , 'votes' ,'Action', 'Adventure', 
'Comedy', 'Crime', 'Family', 'Fantasy', 'Mystery', 'Sci-Fi', 'Thriller']].copy()

# Drop the rows with null values in the selected columns
with_certificate_df = with_certificate_df.dropna()

print(with_certificate_df.isnull().sum())

# Get the number of rows in with_certificate_df
num_rows = with_certificate_df.shape[0]

print(f"Number of rows in with_certificate_df: {num_rows}")

name           0
year           0
certificate    0
rating         0
votes          0
Action         0
Adventure      0
Comedy         0
Crime          0
Family         0
Fantasy        0
Mystery        0
Sci-Fi         0
Thriller       0
dtype: int64
Number of rows in with_certificate_df: 6599


In [7]:
# Finding how may unique certificate catagories are in the dataframe
unique_certificates = with_certificate_df['certificate'].unique()
print(unique_certificates)

['T' 'M' 'E10+' 'E' 'Unrated' 'AO' 'K-A' 'Not Rated' 'GA' '12' 'TV-14'
 'MA-13' 'PG-13' 'EC' 'Approved' 'G' 'MA-17' 'PG' 'CE' 'Passed' 'TV-MA']


In [8]:
# Another df this time add years
with_year_df = games_df[['name', 'year', 'Action', 'Adventure', 
'Comedy', 'Crime', 'Family', 'Fantasy', 'Mystery', 'Sci-Fi', 'Thriller']].copy()

# Drop the rows with null values in the selected columns
with_year_df = with_year_df.dropna()

print(with_year_df.isnull().sum())

# Get the number of rows in with_certificate_df
num_rows = with_year_df.shape[0]

print(f"Number of rows in with_year_df: {num_rows}")

name         0
year         0
Action       0
Adventure    0
Comedy       0
Crime        0
Family       0
Fantasy      0
Mystery      0
Sci-Fi       0
Thriller     0
dtype: int64
Number of rows in with_year_df: 20536


In [9]:
# One last df with year rating and vote
with_ratingvotes_df = games_df[['name', 'year', 'rating', 'votes', 'Action', 'Adventure', 
'Comedy', 'Crime', 'Family', 'Fantasy', 'Mystery', 'Sci-Fi', 'Thriller']].copy()

# Drop the rows with null values in the selected columns
with_ratingvotes_df = with_ratingvotes_df.dropna()

print(with_ratingvotes_df.isnull().sum())

# Get the number of rows in with_certificate_df
num_rows = with_ratingvotes_df.shape[0]

print(f"Number of rows in with_ratingvotes_df: {num_rows}")
print(with_ratingvotes_df.dtypes)
with_ratingvotes_df.head()

name         0
year         0
rating       0
votes        0
Action       0
Adventure    0
Comedy       0
Crime        0
Family       0
Fantasy      0
Mystery      0
Sci-Fi       0
Thriller     0
dtype: int64
Number of rows in with_ratingvotes_df: 11599
name          object
year         float64
rating       float64
votes        float64
Action          bool
Adventure       bool
Comedy          bool
Crime           bool
Family          bool
Fantasy         bool
Mystery         bool
Sci-Fi          bool
Thriller        bool
dtype: object


,name,year,rating,votes,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller
0,Spider-Man,2018.0,9.2,20759.0,True,True,False,False,False,True,False,False,False
1,Red Dead Redemption II,2018.0,9.7,35703.0,True,True,False,True,False,False,False,False,False
2,Grand Theft Auto V,2013.0,9.5,59986.0,True,False,False,True,False,False,False,False,False
3,God of War,2018.0,9.6,26118.0,True,True,False,False,False,False,False,False,False
4,Uncharted 4: A Thief's End,2016.0,9.5,28722.0,True,True,False,False,False,False,False,False,False


I think "with_ratingvotes_df" might be the best for the clustering, at least the first attempt. 

In [10]:
with_ratingvotes_df.to_csv('./Data/with_ratingvotes.csv', index=False)
genreonly_df.to_csv('./Data/genreonly.csv', index=False)
with_certificate_df.to_csv('./Data/with_certificate.csv', index=False)